In [1]:
import numpy as np
import h5py
from progressbar import ProgressBar
import random

f_video_features = h5py.File('/imatge/amontes/work/datasets/ActivityNet/v1.3/features_dataset/features_dataset.hdf5')
f_audio_features = h5py.File('./audio_descriptors.hdf5')
f_output = h5py.File('/imatge/amontes/work/datasets/ActivityNet/v1.3/output_dataset/output_dataset.hdf5')

subset = 'validation'
max_nb_videos = None

batch_size = 256
timesteps = 20

features_size = 4096
output_size = 201
mfcc_size = 80
spec_size = 8

In [2]:
def to_categorical(y, nb_classes=None):
    '''Convert class vector (integers from 0 to nb_classes)
    to binary class matrix, for use with categorical_crossentropy.
    '''
    if not nb_classes:
        nb_classes = np.max(y)+1
    Y = np.zeros((len(y), nb_classes))
    for i in range(len(y)):
        Y[i, y[i]] = 1.
    return Y

In [3]:
videos = f_video_features[subset].keys()

videos_not_features = []
for v in videos:
    if v not in f_audio_features['mfcc'].keys():
        videos_not_features.append(v)
    elif v not in f_audio_features['spec'].keys():
        videos_not_features.append(v)
        
videos_not_features = list(set(videos_not_features))
for vid in videos_not_features:
    videos.remove(vid)

random.shuffle(videos)
if max_nb_videos and max_nb_videos < len(videos):
    videos = videos[:max_nb_videos]

nb_videos = len(videos)
print('Number of videos: {}'.format(nb_videos))

Number of videos: 4675


In [4]:
sequence_stack = []
for _ in range(batch_size):
    sequence_stack.append([])
nb_clips_stack = np.zeros(batch_size).astype(np.int64)
accumulative_clips_stack = []
for _ in range(batch_size):
    accumulative_clips_stack.append([])

for video_id in videos:
    min_pos = np.argmin(nb_clips_stack)
    sequence_stack[min_pos].append(video_id)
    nb_clips_stack[min_pos] += f_video_features[subset][video_id].shape[0]
    accumulative_clips_stack[min_pos].append(nb_clips_stack[min_pos])

min_sequence = np.min(nb_clips_stack)
max_sequence = np.max(nb_clips_stack)
nb_batches_long = max_sequence // timesteps + 1
nb_batches = min_sequence // timesteps
print('Number of batches: {}'.format(nb_batches))

Number of batches: 185


In [5]:
video_features = np.zeros((nb_batches_long*batch_size*timesteps, features_size))
mfcc_features = np.zeros((nb_batches_long*batch_size*timesteps, mfcc_size))
spec_features = np.zeros((nb_batches_long*batch_size*timesteps, spec_size))
output = np.zeros((nb_batches_long*batch_size*timesteps, output_size))
index = np.arange(nb_batches_long*batch_size*timesteps)

progbar = ProgressBar(max_value=batch_size)

for i in range(batch_size):
    batch_index = index // timesteps % batch_size == i
    progbar.update(i)

    pos = 0
    for video_id in sequence_stack[i]:
        # Video features
        vid_features = f_video_features[subset][video_id][...]
        assert vid_features.shape[1] == features_size
        nb_instances = vid_features.shape[0]
        # MFCC features
        mfcc_feat = f_audio_features['mfcc'][video_id][...]
        assert mfcc_feat.shape == (nb_instances, mfcc_size)
        # Spec features
        spec_feat = f_audio_features['spec'][video_id][...]
        assert spec_feat.shape == (1, spec_size), spec_feat.shape
        spec_feat = np.broadcast_to(spec_feat, (nb_instances, spec_size))
        # Output
        output_classes = f_output[subset][video_id][...]
        assert nb_instances == output_classes.shape[0]
        

        video_index = index[batch_index][pos:pos+nb_instances]
        video_features[video_index,:] = vid_features
        mfcc_features[video_index,:] = mfcc_feat
        spec_features[video_index,:] = spec_feat
        output[video_index] = to_categorical(output_classes, nb_classes=output_size)

        pos += nb_instances
        
progbar.finish()
    
f_video_features.close()
f_audio_features.close()
f_output.close()

video_features = video_features[:nb_batches*batch_size*timesteps,:]
assert np.all(np.any(video_features, axis=1))
video_features = video_features.reshape((nb_batches*batch_size, timesteps, features_size))

mfcc_features = mfcc_features[:nb_batches*batch_size*timesteps,:]
assert np.all(np.any(mfcc_features, axis=1))
mfcc_features = mfcc_features.reshape((nb_batches*batch_size, timesteps, mfcc_size))

spec_features = spec_features[:nb_batches*batch_size*timesteps,:]
assert np.all(np.any(spec_features, axis=1))
spec_features = spec_features.reshape((nb_batches*batch_size, timesteps, spec_size))

output = output[:nb_batches*batch_size*timesteps,:]
assert np.all(np.any(output, axis=1))
output = output.reshape((nb_batches*batch_size, timesteps, output_size))

Putting data on position 0 of each batch


  0% (  1 of 256) |                        | Elapsed Time: 0:00:01 ETA:  0:08:18

Putting data on position 1 of each batch


  0% (  2 of 256) |                        | Elapsed Time: 0:00:03 ETA:  0:08:30

Putting data on position 2 of each batch


...

 99% (254 of 256) |####################### | Elapsed Time: 0:07:48 ETA:  0:00:00

Putting data on position 254 of each batch


 99% (255 of 256) |####################### | Elapsed Time: 0:07:50 ETA:  0:00:00

Putting data on position 255 of each batch


100% (256 of 256) |########################| Elapsed Time: 0:07:52 Time: 0:07:52


Computing sample weights

In [6]:
if subset == 'training':
    background_weight = 0.6
    sample_weights = np.ones(output.shape[:2])
    sample_weights[output[:,:,0] == 1] = background_weight

In [7]:
f_dataset = h5py.File('stateful_dataset_with_audio.hdf5', 'r+')
f_dataset_subset = f_dataset.create_group(subset)

f_dataset_subset.create_dataset('vid_features', data=video_features, chunks=(4, timesteps, features_size), dtype='float32')
f_dataset_subset.create_dataset('mfcc_features', data=mfcc_features, chunks=(256, timesteps, mfcc_size), dtype='float32')
f_dataset_subset.create_dataset('spec_features', data=spec_features, chunks=(256, timesteps, spec_size), dtype='float32')
f_dataset_subset.create_dataset('output', data=output, chunks=(256, timesteps, output_size), dtype='float32')
if subset == 'training':
    f_dataset_subset.create_dataset('sample_weight', data=sample_weights, chunks=(256, timesteps), dtype='float32')
f_dataset.close()